In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
print(spark)

In [0]:
# a)	Create Airline Pysaprk sql dataFrame. 
from pyspark.sql.functions import *
import pyspark.sql.functions as func 
from pyspark.sql.functions import stddev
flights_df = spark.read.csv("dbfs:/FileStore/tables/Flights_Delay.csv",header = True , inferSchema = True)
flights_df.show()

+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
| ID|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+-----------

In [0]:
#b)	Describe the table schema & show 10 rows of Dataset

flights_df.show(10)

+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
| ID|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+-----------

In [0]:
# Creating Temp View Table
flights_df.createOrReplaceTempView("flightData")

In [0]:
# c)	Average arrival delay caused by airlines 
flights_df.groupBy('AIRLINE').mean('ARRIVAL_DELAY').show()

+-------+------------------+
|AIRLINE|avg(ARRIVAL_DELAY)|
+-------+------------------+
|     UA| 6.697221614526362|
|     NK|14.206426484907498|
|     AA| 8.386631979187513|
|     EV|10.884270870655678|
|     B6| 13.95852534562212|
|     DL|2.8144726712856043|
|     OO|10.154792043399638|
|     F9|24.103448275862068|
|     US| 5.977315185481719|
|     MQ|19.231592604605904|
|     HA| 4.072423398328691|
|     AS|-1.531766200762389|
|     VX| 5.128571428571429|
|     WN| 3.697840458351697|
+-------+------------------+



In [0]:
# d ) Days of months with average Arrival delays
flights_df.groupBy('MONTH').mean('ARRIVAL_DELAY').show()

+-----+------------------+
|MONTH|avg(ARRIVAL_DELAY)|
+-----+------------------+
|    1|5.8609426229508195|
|    3|11.343454521241178|
|    2| 8.122951948230758|
+-----+------------------+



In [0]:
flights_df.createOrReplaceTempView("flightData")

In [0]:
spark.sql("select MONTH, AVG(ARRIVAL_DELAY) as avgDelay from flightData group by MONTH  ").show()

+-----+------------------+
|MONTH|          avgDelay|
+-----+------------------+
|    1|5.8609426229508195|
|    3|11.343454521241178|
|    2| 8.122951948230758|
+-----+------------------+



In [0]:
# e)	Days of months with median Departure delays.

flights_df.groupBy("MONTH").agg(func.percentile_approx("DEPARTURE_DELAY", 0.5).alias("median")).show()

+-----+------+
|MONTH|median|
+-----+------+
|    1|    -1|
|    3|     0|
|    2|    -1|
+-----+------+



In [0]:
median = spark.sql("select MONTH , percentile_approx(DEPARTURE_DELAY, 0.5) as median from flightData group by MONTH")
median.show()

+-----+------+
|MONTH|median|
+-----+------+
|    1|    -1|
|    3|     0|
|    2|    -1|
+-----+------+



In [0]:
# f)	Arrange weekdays with respect to the average arrival delays

spark.sql("select DAY_OF_WEEK, AVG(ARRIVAL_DELAY) as avg_arrival_Delay from flightData  group by DAY_OF_WEEK order by DAY_OF_WEEK desc").show()

+-----------+------------------+
|DAY_OF_WEEK| avg_arrival_Delay|
+-----------+------------------+
|          7|10.110840438489646|
|          6| 4.888689138576779|
|          5| 6.010538373424971|
|          4| 7.174969021065675|
|          3| 5.587079407806191|
|          2| 8.033644102148358|
|          1|10.807447207297264|
+-----------+------------------+



In [0]:
# g)	Show Analysis for each month with total number of cancellations. (Descending).

spark.sql("select  MONTH, sum(CANCELLED) as totalnoofcancelled  from flightData group by  MONTH order by MONTH DESC ").show()

+-----+------------------+
|MONTH|totalnoofcancelled|
+-----+------------------+
|    3|               456|
|    2|              1087|
|    1|               608|
+-----+------------------+



In [0]:
# i)	Show the airlines that make the maximum number of cancellations.
spark.sql("select  AIRLINE , count(CANCELLED) as maximunnoofcancelledfilghts from flightData where CANCELLED > 0 group by AIRLINE").show()

+-------+---------------------------+
|AIRLINE|maximunnoofcancelledfilghts|
+-------+---------------------------+
|     UA|                        122|
|     NK|                         21|
|     AA|                        241|
|     EV|                        312|
|     B6|                        145|
|     DL|                        177|
|     OO|                        153|
|     F9|                         11|
|     US|                        169|
|     MQ|                        414|
|     HA|                          3|
|     AS|                         12|
|     VX|                         13|
|     WN|                        358|
+-------+---------------------------+



In [0]:
# j)	Find and order airlines in descending that make the most number of diversions.

spark.sql("select AIRLINE , sum(DIVERTED) as divertedflights from flightData where DIVERTED > 0 group by AIRLINE order by AIRLINE desc").show()

+-------+---------------+
|AIRLINE|divertedflights|
+-------+---------------+
|     WN|             35|
|     US|              9|
|     UA|              8|
|     OO|             25|
|     MQ|              5|
|     HA|              1|
|     EV|             22|
|     DL|             18|
|     B6|             16|
|     AA|             12|
+-------+---------------+



In [0]:
# k)	Show days of month that seen the most number of diversion.

spark.sql("select MONTH as day_of_month , sum(DIVERTED) as divertedflights from flightData where DIVERTED > 0 group by MONTH order by divertedflights desc").show()

+------------+---------------+
|day_of_month|divertedflights|
+------------+---------------+
|           2|             60|
|           1|             56|
|           3|             35|
+------------+---------------+



In [0]:
# m)	Calculate mean and standard deviation of arrival delay for all flights in minutes.

df2 = spark.sql("select round(stddev(ARRIVAL_DELAY)*60/60 , 0) as StdDev_ARR_Delay_in_Min  , round(mean(ARRIVAL_DELAY)*60/60 , 0) as mean_ARR_Delay_in_Min from flightData  ")

df2.show()

+-----------------------+---------------------+
|StdDev_ARR_Delay_in_Min|mean_ARR_Delay_in_Min|
+-----------------------+---------------------+
|                   42.0|                  8.0|
+-----------------------+---------------------+



In [0]:
# l)	Calculate mean and standard deviation of departure delay for all flights in minutes.

df2 = spark.sql("select round(stddev(DEPARTURE_DELAY)*60/60 , 0) as stddev_DEP_DELAY_in_min  , round(mean(DEPARTURE_DELAY)*60/60 , 0) as mean_DEP_DELAY_in_min from flightData  ")

df2.show()

+-----------------------+---------------------+
|stddev_DEP_DELAY_in_min|mean_DEP_DELAY_in_min|
+-----------------------+---------------------+
|                   40.0|                 11.0|
+-----------------------+---------------------+



In [0]:
spark.sql("select MONTH ,sum(DIVERTED)   from flightData GROUP BY MONTH").show(100)

+-----+-------------+
|MONTH|sum(DIVERTED)|
+-----+-------------+
|    1|           56|
|    3|           35|
|    2|           60|
+-----+-------------+



In [0]:
# n)	Find all diverted Route from a source to destination Airport & which route is the most diverted.

spark.sql("select ORIGIN_AIRPORT,DESTINATION_AIRPORT, count(DIVERTED) as Total_Diversion from flightData group by ORIGIN_AIRPORT,DESTINATION_AIRPORT order by Total_Diversion desc").show(5)

+--------------+-------------------+---------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|Total_Diversion|
+--------------+-------------------+---------------+
|           LAX|                JFK|            146|
|           LAX|                SFO|            142|
|           JFK|                LAX|            137|
|           SFO|                LAX|            131|
|           LAS|                LAX|            123|
+--------------+-------------------+---------------+
only showing top 5 rows



In [0]:
# h)	Show the top 10 busiest airport. [Hint: Sum Total of Arrival and Departures from an Airport].

spark.sql("select ORIGIN_AIRPORT as Aiports, count(ORIGIN_AIRPORT)+count(DESTINATION_AIRPORT) as cnt from flightData group by ORIGIN_AIRPORT order by cnt DESC").show(10)

+-------+----+
|Aiports| cnt|
+-------+----+
|    ATL|7154|
|    ORD|5792|
|    DFW|5602|
|    LAX|4094|
|    DEN|4070|
|    IAH|3224|
|    PHX|3178|
|    SFO|3010|
|    LAS|2832|
|    MCO|2400|
+-------+----+
only showing top 10 rows

